In [1]:
import csv
import json
from os.path import join
import requests

Path to CSV file (download from https://github.com/swiss-art-research-net/bso-vikus-viewer-data/blob/main/data/images.csv)

In [2]:
imageCSV = '../training/images.csv'

Path to JSON file containing classifications, created with [RunwayML](https://runwayml.com/)

In [3]:
inputFile = '../training/Annotation Group - Images and Colour Bars for BSO Examples.json'

Directory to download images into

In [4]:
fileDir = '../training/data/images/'

Download images

In [5]:
imageURLs = {}
with open(imageCSV, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        imageURLs[row['id'] + '.jpg'] = row['image']

In [6]:
with open(inputFile, 'r') as f:
    data = json.load(f)

In [7]:
maxRetries = 5
for file in data['files'].keys():
    outputFile = join(fileDir, file)
    url = imageURLs[file]
    r = requests.get(url, allow_redirects=True)
    retries = 1
    while not 'image' in r.headers['Content-Type'] and retries <= maxRetries:
        # Try again if no image comes back
        time.sleep(1)
        r = requests.get(url, allow_redirects=True)
        retries += 1

    if retries >= maxRetries:
        print("Could not download", id, url)
    else:
        with open(outputFile, 'wb') as f:
            f.write(r.content)

In [8]:
imageURLs

{'nb-1000029.jpg': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/CH-NB_-_Z%C3%BCrich%2C_von_der_obern_Strasse_gegen_den_H%C3%BCtliberg_aufgenommen_%3D_Z%C3%BCrich%2C_prise_de_la_route_sup%C3%A9rieure_vers_le_H%C3%BCtliberg_-_Collection_Gugelmann_-_GS-GUGE-83-7-1.tif&width=1024',
 'nb-1000030.jpg': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/CH-NB_-_Das_Rathhaus_u._das_Kaffeehaus_zur_Saffran_%3D_L%27H%C3%B4tel_de_ville_et_le_Caf%C3%A9_Safran_-Randvignette_oben_links-_-_Collection_Gugelmann_-_GS-GUGE-83-7-2.tif&width=1024',
 'nb-1000031.jpg': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/CH-NB_-_Der_Wellenberg_gegen_das_Stadthaus_%3D_Le_Wellenberg_vers_la_maison_de_ville_-Randvignette_oben_Mitte_links-_-_Collection_Gugelmann_-_GS-GUGE-83-7-3.tif&width=1024',
 'nb-1000032.jpg': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/CH-NB_-_Salomon_Gessners_Denkmal_%3D_Le_monument_de_Salomon_Gessne